In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import pathlib
import cv2

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
directory = "/kaggle/input/mvtec-ad"
root = pathlib.Path(directory)
classes = sorted([p.name for p in root.iterdir() if p.is_dir()])
print("Detected classes:", classes)

In [ ]:
filepaths, labels = [], []
for cls in classes:
    for img_path in (root / cls).rglob("*"):
        if img_path.suffix.lower() in [".png", ".jpg", ".jpeg"]:
            filepaths.append(str(img_path))
            labels.append(cls)

print("Total images:", len(filepaths))

In [ ]:
train_paths, test_paths, train_labels, test_labels = train_test_split(
    filepaths, labels, test_size=0.2, random_state=42, stratify=labels
)
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)
print("Train:", len(train_paths), "Val:", len(val_paths), "Test:", len(test_paths))

In [ ]:
class_names = classes
class_to_index = {c: i for i, c in enumerate(class_names)}
num_classes = len(class_names)

In [ ]:
def load_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

def augment(img, label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, 0.08)
    img = tf.image.random_contrast(img, 0.9, 1.1)
    return img, label

def paths_to_dataset(paths, labels, shuffle=False, augment_data=False):
    labels_idx = [class_to_index[l] for l in labels]
    ds = tf.data.Dataset.from_tensor_slices((paths, labels_idx))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths))
    ds = ds.map(lambda p, l: load_and_preprocess(p, l), num_parallel_calls=AUTOTUNE)
    if augment_data:
        ds = ds.map(augment, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

In [ ]:
train_ds = paths_to_dataset(train_paths, train_labels, shuffle=True, augment_data=True)
val_ds = paths_to_dataset(val_paths, val_labels)
#test_ds = paths_to_dataset(test_paths, test_labels)
#train_ds
data_dir = directory
test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    label_mode="int",   # integer labels
    image_size=(224, 224),
    shuffle=False
)


In [ ]:
base = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base.trainable = False

inputs = layers.Input(shape=(224, 224, 3))
x = preprocess_input(inputs)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = Model(inputs, outputs)
model.compile(optimizer=optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

In [ ]:
cb = [
    callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss"),
    callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]

In [ ]:
history1 = model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=cb)

In [ ]:
for layer in base.layers[-30:]:
    layer.trainable = True

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",  # <-- use sparse
    metrics=["accuracy"]
)

history2 = model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=cb)





In [ ]:
model.load_weights("best_model.h5")
y_true, y_pred = [], []
for imgs, labels in test_ds:
    preds = model.predict(imgs, verbose=0)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(labels.numpy())

print(classification_report(y_true, y_pred, target_names=class_names))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))
acc = accuracy_score(y_true, y_pred)
print("Accuracy score:", acc)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths
data_dir = directory  # your dataset root

# Data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 20% data for validation
)

# Train generator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",  # <-- one-hot labels
    subset="training"
)

# Validation generator
val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Optional: separate test split (if you want an unseen set)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=False  # important for evaluation
)


In [ ]:
#show_gradcam(test_paths[0], model, layer="resnet50_conv5_block3_out")
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)
print("Test Loss:", test_loss)


In [ ]:
model.save("medicinal_leaf_model.h5")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# path to your test image
img_path = "/kaggle/input/mvtec-ad/pill/test/color/000.png"
# Load and preprocess
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
img_array = img_array / 255.0  # normalize same as training


In [ ]:
pred = model.predict(img_array)
pred_class = np.argmax(pred, axis=1)[0]

print("Predicted class index:", pred_class)


In [ ]:
# If you used ImageDataGenerator
class_indices = train_generator.class_indices  # dictionary
classes = list(class_indices.keys())

# OR if you used image_dataset_from_directory
#classes = train_ds.class_names

print("Predicted class:", classes[pred_class])


In [ ]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np

# Root dataset folder (update this to your dataset location)
dataset_dir = "/kaggle/input/mvtec-ad"

# Get class names in the same order as training
# If you used ImageDataGenerator
classes = list(train_generator.class_indices.keys())
# OR if you used image_dataset_from_directory
#classes = train_ds.class_names

# Loop through each top-level category (bottle, capsule, cable, ...)
categories = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

for category in categories:
    test_dir = os.path.join(dataset_dir, category, "test")
    if os.path.exists(test_dir):
        print(f"\n=== Category: {category} ===")
        
        # Loop through each defect type inside test (e.g. "good", "broken_large", etc.)
        sub_classes = [d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))]
        
        for sub in sub_classes:
            sub_path = os.path.join(test_dir, sub)
            image_files = [f for f in os.listdir(sub_path) if f.endswith((".png", ".jpg", ".jpeg"))]
            
            # Just test the first image for demonstration
            if image_files:
                img_path = os.path.join(sub_path, image_files[0])
                
                # Load and preprocess
                img = image.load_img(img_path, target_size=(224, 224))
                img_array = image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0) / 255.0
                
                # Predict
                pred = model.predict(img_array, verbose=0)
                pred_class = np.argmax(pred, axis=1)[0]
                
                print(f"  Subclass: {sub:20s} → Predicted: {classes[pred_class]}")


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

# Root dataset folder
dataset_dir = "/kaggle/input/mvtec-ad"

# Class names (from your training set)
#classes = train_ds.class_names  
# or 
classes= list(train_generator.class_indices.keys())


# Loop through categories
categories = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

for category in categories:
    test_dir = os.path.join(dataset_dir, category, "test")
    if os.path.exists(test_dir):
        print(f"\n=== Category: {category} ===")

        sub_classes = [d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))]
        
        for sub in sub_classes:
            sub_path = os.path.join(test_dir, sub)
            image_files = [f for f in os.listdir(sub_path) if f.endswith((".png", ".jpg", ".jpeg"))]

            # Show up to 12 images in a grid (4x3)
            n_show = min(12, len(image_files))
            if n_show == 0:
                continue

            plt.figure(figsize=(12, 9))
            for i, img_file in enumerate(image_files[:n_show]):
                img_path = os.path.join(sub_path, img_file)

                # Preprocess image
                img = image.load_img(img_path, target_size=(224, 224))
                img_array = image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0) / 255.0

                # Predict
                pred = model.predict(img_array, verbose=0)
                pred_class = np.argmax(pred, axis=1)[0]

                # Plot
                plt.subplot(3, 4, i+1)  # 3 rows x 4 cols
                plt.imshow(image.load_img(img_path))
                plt.title(f"True: {sub}\nPred: {classes[pred_class]}", fontsize=9)
                plt.axis("off")

            plt.suptitle(f"{category} / {sub}", fontsize=12, weight="bold")
            plt.tight_layout()
            plt.show()


In [ ]:
import random
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import os

# Path to a folder with images (not your dataset)
random_img_folder = "/kaggle/input/my-random-images1"  # change this to your folder

# Collect all image file paths
all_images = []
for root, dirs, files in os.walk(random_img_folder):
    for file in files:
        if file.lower().endswith((".png", ".jpg", ".jpeg")):
            all_images.append(os.path.join(root, file))

# Pick a random image
img_path = random.choice(all_images)

# Load and show
img = image.load_img(img_path)  # no resizing
plt.imshow(img)
plt.axis("off")
plt.title("Random Image")
plt.show()
